# easysnmp

In [13]:
import easysnmp

In [14]:
session = easysnmp.Session(hostname='localhost', community='public', version=2)

In [15]:
location = session.get('sysLocation.0')
location.oid, location.oid_index, location.snmp_type, location.value

('sysLocation', '0', 'OCTETSTR', 'Unknown (edit /etc/snmp/snmpd.conf)')

In [16]:
iftable = session.walk('IF-MIB::ifTable')

In [17]:
for item in iftable:
    print(item.oid, item.oid_index, item.snmp_type, item.value, len(item.value))

ifIndex 1 INTEGER 1 1
ifIndex 2 INTEGER 2 1
ifIndex 3 INTEGER 3 1
ifIndex 4 INTEGER 4 1
ifIndex 5 INTEGER 5 1
ifDescr 1 OCTETSTR lo 2
ifDescr 2 OCTETSTR enp4s0 6
ifDescr 3 OCTETSTR wlp0s18f2u5 11
ifDescr 4 OCTETSTR docker0 7
ifDescr 5 OCTETSTR br-f863e3fde3cb 15
ifType 1 INTEGER 24 2
ifType 2 INTEGER 6 1
ifType 3 INTEGER 6 1
ifType 4 INTEGER 6 1
ifType 5 INTEGER 6 1
ifMtu 1 INTEGER 65536 5
ifMtu 2 INTEGER 1500 4
ifMtu 3 INTEGER 1500 4
ifMtu 4 INTEGER 1500 4
ifMtu 5 INTEGER 1500 4
ifSpeed 1 GAUGE 10000000 8
ifSpeed 2 GAUGE 0 1
ifSpeed 3 GAUGE 0 1
ifSpeed 4 GAUGE 0 1
ifSpeed 5 GAUGE 0 1
ifPhysAddress 1 OCTETSTR  0
ifPhysAddress 2 OCTETSTR oeÒ 6
ifPhysAddress 3 OCTETSTR tÚ8ð 6
ifPhysAddress 4 OCTETSTR BÕW$ß 6
ifPhysAddress 5 OCTETSTR B(B 6
ifAdminStatus 1 INTEGER 1 1
ifAdminStatus 2 INTEGER 1 1
ifAdminStatus 3 INTEGER 1 1
ifAdminStatus 4 INTEGER 1 1
ifAdminStatus 5 INTEGER 1 1
ifOperStatus 1 INTEGER 1 1
ifOperStatus 2 INTEGER 2 1
ifOperStatus 3 INTEGER 1 1
ifOperStatus 4 INTEGER 

The type of ``.value`` is always a Python string but the string returned in ``.snmp_type`` can be used to convert to the correct Python type. T
* INTEGER32
* INTEGER
* UNSIGNED32
* GAUGE
* IPADDR
* OCTETSTR
* TICKS
* OPAQUE
* OBJECTID
* NETADDR
* COUNTER64
* NULL
* BITS
* UINTEGER

This is unfortunately not always straight-forward as only the base types are used. For example ``ifPhysAddress`` is returned as an ``OCTETSTR`` which must then be converted into the proper hexadecimal representation of a layer-2 address.

In [83]:
macaddress = session.get('IF-MIB::ifPhysAddress.2')

In [84]:
':'.join(['%02x' % (ord(x)) for x in macaddress.value])

'1c:6f:65:8d:9a:d2'

# easysnmptable

In [85]:
import easysnmptable

In [86]:
session = easysnmptable.Session(hostname='localhost', community='public', version=2)

In [87]:
iftable = session.gettable('IF-MIB::ifTable')

In [88]:
iftable.indices

{'1', '2', '3', '4', '5'}

In [89]:
iftable.cols

{'ifAdminStatus',
 'ifDescr',
 'ifInDiscards',
 'ifInErrors',
 'ifInNUcastPkts',
 'ifInOctets',
 'ifInUcastPkts',
 'ifInUnknownProtos',
 'ifIndex',
 'ifLastChange',
 'ifMtu',
 'ifOperStatus',
 'ifOutDiscards',
 'ifOutErrors',
 'ifOutNUcastPkts',
 'ifOutOctets',
 'ifOutQLen',
 'ifOutUcastPkts',
 'ifPhysAddress',
 'ifSpecific',
 'ifSpeed',
 'ifType'}

In [69]:
import pprint
for index,row in iftable.rows.items():
    pprint.pprint(index)
    pprint.pprint(row)

'1'
{'ifAdminStatus': '1',
 'ifDescr': 'lo',
 'ifInDiscards': '0',
 'ifInErrors': '0',
 'ifInNUcastPkts': '0',
 'ifInOctets': '6369246',
 'ifInUcastPkts': '4677',
 'ifInUnknownProtos': '0',
 'ifIndex': '1',
 'ifLastChange': '0',
 'ifMtu': '65536',
 'ifOperStatus': '1',
 'ifOutDiscards': '0',
 'ifOutErrors': '0',
 'ifOutNUcastPkts': '0',
 'ifOutOctets': '6369246',
 'ifOutQLen': '0',
 'ifOutUcastPkts': '4677',
 'ifPhysAddress': '',
 'ifSpecific': '.0.0',
 'ifSpeed': '10000000',
 'ifType': '24'}
'2'
{'ifAdminStatus': '1',
 'ifDescr': 'enp4s0',
 'ifInDiscards': '0',
 'ifInErrors': '0',
 'ifInNUcastPkts': '0',
 'ifInOctets': '0',
 'ifInUcastPkts': '0',
 'ifInUnknownProtos': '0',
 'ifIndex': '2',
 'ifLastChange': '0',
 'ifMtu': '1500',
 'ifOperStatus': '2',
 'ifOutDiscards': '0',
 'ifOutErrors': '0',
 'ifOutNUcastPkts': '0',
 'ifOutOctets': '0',
 'ifOutQLen': '0',
 'ifOutUcastPkts': '0',
 'ifPhysAddress': '\x1coe\x8d\x9aÒ',
 'ifSpecific': '.0.0',
 'ifSpeed': '0',
 'ifType': '6'}
'3'
{'ifAdmi

A short-coming of **easysnmptable** is that it does not return the ``snmp_type`` for columns. A possible work-around is to use **easysnmp** to fetch a single column for a random index. This should probably be memoized or cached.

In [72]:
random_index = iftable.indices.pop()
iftable.indices.add(random_index)
random_index

'3'

In [76]:
column2type = {column: session.get('{}.{}'.format(column, random_index)).snmp_type for column in iftable.cols}
column2type

{'ifLastChange': 'TICKS',
 'ifOutNUcastPkts': 'COUNTER',
 'ifInDiscards': 'COUNTER',
 'ifOutOctets': 'COUNTER',
 'ifInErrors': 'COUNTER',
 'ifPhysAddress': 'OCTETSTR',
 'ifSpeed': 'GAUGE',
 'ifSpecific': 'OBJECTID',
 'ifOutQLen': 'GAUGE',
 'ifDescr': 'OCTETSTR',
 'ifInOctets': 'COUNTER',
 'ifOutUcastPkts': 'COUNTER',
 'ifAdminStatus': 'INTEGER',
 'ifOperStatus': 'INTEGER',
 'ifType': 'INTEGER',
 'ifIndex': 'INTEGER',
 'ifInNUcastPkts': 'COUNTER',
 'ifInUcastPkts': 'COUNTER',
 'ifInUnknownProtos': 'COUNTER',
 'ifOutDiscards': 'COUNTER',
 'ifOutErrors': 'COUNTER',
 'ifMtu': 'INTEGER'}

Such a table could be build by "walking" a device.

In [82]:
column2type = {item.oid: item.snmp_type for item in session.walk('IF-MIB::ifTable')}
column2type

{'ifIndex': 'INTEGER',
 'ifDescr': 'OCTETSTR',
 'ifType': 'INTEGER',
 'ifMtu': 'INTEGER',
 'ifSpeed': 'GAUGE',
 'ifPhysAddress': 'OCTETSTR',
 'ifAdminStatus': 'INTEGER',
 'ifOperStatus': 'INTEGER',
 'ifLastChange': 'TICKS',
 'ifInOctets': 'COUNTER',
 'ifInUcastPkts': 'COUNTER',
 'ifInNUcastPkts': 'COUNTER',
 'ifInDiscards': 'COUNTER',
 'ifInErrors': 'COUNTER',
 'ifInUnknownProtos': 'COUNTER',
 'ifOutOctets': 'COUNTER',
 'ifOutUcastPkts': 'COUNTER',
 'ifOutNUcastPkts': 'COUNTER',
 'ifOutDiscards': 'COUNTER',
 'ifOutErrors': 'COUNTER',
 'ifOutQLen': 'GAUGE',
 'ifSpecific': 'OBJECTID'}